In [0]:
%run 
./_common_helpers

---
### Purpose: ###  
Feed raw 835 remittance bronze data into silver `payments_835_events` and `payments_835_by_claim` tables


---
- Normalize types  
- Join payer and denial reason dimensions for consistent naming and grouping  
- Deduplicate payment events (keep latest by ingest timestamp / file)  
- Bad rows missing `claim_id`, invalid `payment_date`, or non-numeric amounts go to the `payments_835_rejects` table  
- Roll up payments by claim to provide quick join targets for EHR claim headers  
---


---
### Assumes: ###  
Each 835 payment row represents one claim-level remittance event (not line-item).  
Future line-level 835 detail, if ingested, will remain compatible with the current structure.  

In [0]:
USE CATALOG claims360_dev;
USE SCHEMA silver;
-------------------------------------------------------------------------------
--           1) Staging: typed, validated, dedup
-------------------------------------------------------------------------------

CREATE OR REPLACE TEMP VIEW stage_payments_835 AS
WITH base AS (
  SELECT
    upper(trim(claim_id))                AS claim_id,
    upper(trim(payer_id))                AS payer_id,
    upper(trim(patient_id))              AS patient_id,
    upper(trim(remit_id))                AS remit_id,
    upper(trim(payer_control_number))    AS payer_control_number,
    upper(trim(check_or_eft_trace))      AS check_or_eft_trace,
    upper(trim(adjustment_reason_code))  AS adjustment_reason_code,
    trim(payer_name)                     AS payer_name_raw,

    -- Type conversions
    silver.fn_to_ts_safe(payment_date)   AS payment_ts,   -- payment timestamp
    silver.fn_to_date_safe(payment_date) AS payment_date, 
    silver.fn_to_date_safe(service_date) AS service_date,
    silver.fn_to_dec_safe(payment_amount)     AS payment_amount,
    silver.fn_to_dec_safe(adjustment_amount)  AS adjustment_amount,

    -- Normalizations
    CASE WHEN check_or_eft_trace LIKE 'EFT%' THEN 'EFT'
         WHEN check_or_eft_trace IS NOT NULL THEN 'CHECK_OR_OTHER'
         ELSE NULL END AS payment_method,

    _ingest_ts, _ingest_file, _source_system
  FROM bronze.payments_835_raw
),
filtered AS (
  -- Keep only rows with a claim_id, parsable payment_date, and numeric payment_amount.
  -- We’ll send the rest to rejects.
  SELECT *
  FROM base
  WHERE claim_id IS NOT NULL
    AND payment_date IS NOT NULL
    AND payment_amount IS NOT NULL
),
enriched AS (
  SELECT
    f.*,
    -- Join remit code map (category, notes desceription)
    r.reason_category,
    r.notes AS reason_desc,

    -- Join payer dimension for canonical naming/grouping
    p.payer_name       AS payer_name_dim,
    p.payer_group      AS payer_group
  FROM filtered f
  LEFT JOIN dim_denial_reason_map r
    ON upper(trim(r.code)) = f.adjustment_reason_code
  LEFT JOIN dim_payer p
    ON upper(trim(p.payer_id)) = f.payer_id
), 
with_dedupe AS (
  SELECT
    e.*,
    ROW_NUMBER() OVER (
      PARTITION BY remit_id, claim_id, payment_date, payment_amount, adjustment_reason_code, check_or_eft_trace
      ORDER BY _ingest_ts DESC, _ingest_file DESC
    ) AS rn
  FROM enriched e
)
SELECT *
FROM with_dedupe
WHERE rn = 1;


In [0]:

-------------------------------------------------------------------------------
-- 2) Silver target: events table (SCD1)
-------------------------------------------------------------------------------

CREATE OR REPLACE TABLE silver.payments_835_events (
  claim_id STRING NOT NULL,
  patient_id STRING,
  payer_id STRING,
  payer_control_number STRING,
  remit_id STRING,
  check_or_eft_trace STRING,
  payment_method STRING,             
  payment_date DATE NOT NULL,
  payment_ts TIMESTAMP NOT NULL,
  service_date DATE,
  payment_amount DECIMAL(18,2) NOT NULL,
  adjustment_amount DECIMAL(18,2),    
  reason_code STRING,
  reason_category STRING,
  reason_desc STRING,
  payer_name STRING,                  
  payer_group STRING,
  source_system STRING,
  _ingest_ts TIMESTAMP,
  _ingest_file STRING,

  -- Useful computed fields
  net_effect DECIMAL(18,2),           -- payment + adjustment
  is_debit BOOLEAN                   -- true when net_effect < 0

)
USING DELTA
CLUSTER BY AUTO;

In [0]:
-------------------------------------------------------------------------------
-- 3) Type-1 upsert of events
-------------------------------------------------------------------------------

MERGE INTO silver.payments_835_events t
USING (
  SELECT
    claim_id,
    patient_id,
    payer_id,
    payer_control_number,
    remit_id,
    check_or_eft_trace,
    payment_method,
    payment_date,
    payment_ts,
    service_date,
    payment_amount,
    adjustment_amount,
    adjustment_reason_code AS reason_code,
    reason_category,
    reason_desc,
    COALESCE(payer_name_dim, initcap(payer_name_raw)) AS payer_name,
    payer_group,
    _source_system AS source_system,
    _ingest_ts, _ingest_file,
    (COALESCE(payment_amount, 0) + COALESCE(adjustment_amount, 0)) AS net_effect,
    (COALESCE(payment_amount, 0) + COALESCE(adjustment_amount, 0)) < 0 AS is_debit
  FROM stage_payments_835
) s
ON  t.remit_id             = s.remit_id
AND t.claim_id             = s.claim_id
AND t.payment_date         = s.payment_date
AND t.payment_amount       = s.payment_amount
AND t.reason_code          = s.reason_code
AND t.check_or_eft_trace   = s.check_or_eft_trace
WHEN MATCHED AND s._ingest_ts > t._ingest_ts THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;

In [0]:
-------------------------------------------------------------------------------
-- 4) Rejections (observability)
-------------------------------------------------------------------------------

CREATE OR REPLACE TABLE silver.payments_835_rejects AS
SELECT
  r.*,
  CASE
    WHEN claim_id IS NULL THEN 'MISSING_CLAIM_ID'
    WHEN silver.fn_to_date_safe(payment_date) IS NULL THEN 'BAD_PAYMENT_DATE'
    WHEN silver.fn_to_dec_safe(payment_amount) IS NULL THEN 'BAD_PAYMENT_AMOUNT'
    ELSE 'OTHER'
  END AS reject_reason
FROM bronze.payments_835_raw r
WHERE claim_id IS NULL
   OR silver.fn_to_date_safe(payment_date) IS NULL
   OR silver.fn_to_dec_safe(payment_amount) IS NULL;

In [0]:
-------------------------------------------------------------------------------
-- 5) Rollup by claim: quick join target for EHR
-------------------------------------------------------------------------------

CREATE OR REPLACE TABLE silver.payments_835_by_claim AS
SELECT
  claim_id,
  SUM(payment_amount)                                         AS paid_amount_to_date,
  SUM(COALESCE(adjustment_amount, 0))                         AS adjustments_to_date,
  SUM(COALESCE(payment_amount,0) + COALESCE(adjustment_amount,0)) AS net_paid_to_date,
  MAX_BY(remit_id,           payment_ts)                      AS last_remit_id,
  MAX_BY(check_or_eft_trace, payment_ts)                      AS last_trace,
  MAX(payment_date)                                           AS last_payment_date,
  MAX(payment_ts)                                             AS last_payment_ts,
  COLLECT_SET(reason_code)                                    AS reason_codes_seen,
  COUNT(*)                                                    AS payment_event_count
FROM silver.payments_835_events
GROUP BY claim_id;

In [0]:
-------------------------------------------------------------------------------
-- 6) Nudge EHR claim headers with the latest payment time
-------------------------------------------------------------------------------

MERGE INTO silver.claim_headers t
USING (
  SELECT claim_id, last_payment_ts
  FROM silver.payments_835_by_claim
) p
ON t.claim_id = p.claim_id
WHEN MATCHED AND (
      t.last_835_event_ts IS NULL
   OR p.last_payment_ts > t.last_835_event_ts
) THEN UPDATE SET last_835_event_ts = p.last_payment_ts;